<a href="https://colab.research.google.com/github/rajeshmath/ProfTrajanov/blob/master/math_rajesh_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --ignore-installed -q pyspark==3.2.1

     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 198 kB 59.9 MB/s 


In [ ]:
import os
import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:

# requesting Spark context
sc = SparkContext(appName="HW1_Task1")

In [ ]:
#Exception Handling and removing wrong datalines
def isfloat(value):
    try:
        float(value)
        return True
 
    except:
         return False

In [ ]:
#For example, remove lines if they don’t have 16 values and 
# checking if the trip distance and fare amount is a float number
# checking if the trip duration is more than a minute, trip distance is more than 0.1 miles, 
# fare amount and total amount are more than 0.1 dollars
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[4])> 60 and float(p[5])>0.10 and float(p[11])> 0.10 and float(p[16])> 0.10):
                return p



In [ ]:
# Upload the data
# At this point you have to upload your data to google colab
! wget -q https://storage.googleapis.com/cs777_spring_2022_public/taxi-data-sorted-verysmall.csv

In [ ]:
pwd

'/content'

In [ ]:
ls -la

total 1900
drwxr-xr-x 1 root root    4096 Mar 11 15:09 ./
drwxr-xr-x 1 root root    4096 Mar 11 14:44 ../
drwxr-xr-x 4 root root    4096 Mar  9 14:47 .config/
drwxr-xr-x 2 root root    4096 Mar 11 15:09 .ipynb_checkpoints/
drwxr-xr-x 1 root root    4096 Mar  9 14:48 sample_data/
-rw-r--r-- 1 root root 1921037 Mar 11 15:07 taxi-data-sorted-verysmall.csv


In [ ]:
# Set your file path here 
path="file:///content/"

# You have 1 files 

testFile= path + "taxi-data-sorted-verysmall.csv"


testDataFrame = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep =",").load(testFile)

testDataFrame.show()

+--------------------------------+--------------------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+---+----+------+------+------+------+----+
|07290D3599E7A0D62097A346EFCC1FB5|E7750A37CAB07D0DFF0AF7E3573AC141|2013-01-01 00:00:00|2013-01-01 00:02:00|120|0.44|-73.956528|40.716976|-73.962440|40.715008|CSH|3.50|0.5012|0.5013|0.0014|0.0015|4.50|
+--------------------------------+--------------------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+---+----+------+------+------+------+----+
|            22D70BF00EEB0ADC8...|            3FF2709163DE7036F...|2013-01-01 00:02:00|2013-01-01 00:02:00|  0| 0.0|       0.0|      0.0|       0.0|      0.0|CSH|27.0|   0.0|   0.5|   0.0|   0.0|27.5|
|            0EC22AAF491A8BD91...|            778C92B26AE78A9EB...|2013-01-01 00:01:00|2013-01-01 00:03:00|120|0.71|-73.973145|40.752827|-73.965897|40.760445|CSH| 4.0|   0.5|   0.5|   0.0|   0.0| 

In [ ]:
testRDD = testDataFrame.rdd.map(tuple)
testRDD.take(1)

[('22D70BF00EEB0ADC83BA8177BB861991',
  '3FF2709163DE7036FCAA4E5A3324E4BF',
  '2013-01-01 00:02:00',
  '2013-01-01 00:02:00',
  0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  'CSH',
  27.0,
  0.0,
  0.5,
  0.0,
  0.0,
  27.5)]

In [ ]:

# calling isfloat and correctRows functions to cleaning up data
taxilinesCorrected = testRDD.filter(correctRows)
taxilinesCorrected.take(1)

[('0EC22AAF491A8BD91F279350C2B010FD',
  '778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01 00:01:00',
  '2013-01-01 00:03:00',
  120,
  0.71,
  -73.973145,
  40.752827,
  -73.965897,
  40.760445,
  'CSH',
  4.0,
  0.5,
  0.5,
  0.0,
  0.0,
  5.0)]